### Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по правилам:

оценка 2 и ниже — низкий рейтинг;
оценка 4 и ниже — средний рейтинг;
оценка 4.5 и 5 — высокий рейтинг.
Результат классификации запишите в столбец class.

In [27]:
import pandas as pd

In [212]:
%%time

ratings = pd.read_csv('ml-latest-small/ratings.csv', sep=',', encoding='utf-8')
movies = pd.read_csv('ml-latest-small/movies.csv', sep=',', encoding='utf-8')
joined = ratings.merge(movies, on = 'movieId', how = 'left') # объединяем таблицы "ratings" и "movies" по колонке 'movieId'

rating1 = [0.5, 2.0, 4.0, 5.0] # разбиваем на интервалы числовые данные оценки: 0.5-2.0, 2.0-4.0, 4.0-5.0, правое значение входит в интервал (right = True), первое значение (0.5) входит в интервал (include_lowest=True))
class1 = ['Low', 'Average', 'High'] # названия для каждого из трех интервалов соответственно
joined['class_'] = pd.cut(joined['rating'], bins=rating1, labels=class1, right = True, include_lowest=True) # разбиваем на интервалы числовые данные оценки, присваиваем им названия и выносим их в отдельную колонку "class"
joined

CPU times: total: 62.5 ms
Wall time: 69.8 ms


,userId,movieId,rating,timestamp,title,genres,class_
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,Average
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,Average
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,Average
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,Low
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,Average
...,...,...,...,...,...,...,...
99999,671,6268,2.5,1065579370,Raising Victor Vargas (2002),Comedy|Drama|Romance,Average
100000,671,6269,4.0,1065149201,Stevie (2002),Documentary,Average
100001,671,6365,4.0,1070940363,"Matrix Reloaded, The (2003)",Action|Adventure|Sci-Fi|Thriller|IMAX,Average
100002,671,6385,2.5,1070979663,Whale Rider (2002),Drama,Average


In [214]:
# Проверка
print(joined['class_'].unique())
print(joined['rating'].unique())
print(len(joined['movieId'].unique()))
print(len(ratings['movieId'].unique()))

['Average', 'Low', 'High']
Categories (3, object): ['Low' < 'Average' < 'High']
[2.5 3.  2.  4.  3.5 1.  5.  4.5 1.5 0.5]
9066
9066


## Задание 2
Используйте файл keywords.csv.
Нужно написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определённому региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:
geo_data = {
‘Центр’: [‘москва’, ‘тула’, ‘ярославль’],
‘Северо-Запад’: [‘петербург’, ‘псков’, ‘мурманск’],
‘Дальний Восток’: [‘владивосток’, ‘сахалин’, ‘хабаровск’]
}

Результат классификации запишите в отдельный столбец region.

In [288]:
%%time

df = pd.read_csv('ml-latest-small/keywords.csv', sep=',', encoding='utf-8')

import re

def region(row): 
    keyword = row['keyword']
    centre = re.search(r'.*(москва|тула|ярославль)+.*', keyword)
    northwest = re.search(r'.*(петербург|псков|мурманск)+.*', keyword)
    fareast = re.search(r'.*(владивосток|сахалин|хабаровск)+.*', keyword)    
    if centre:
        return 'Centre'
    if northwest:
        return 'Northwest'
    if fareast:
        return 'The Far East'
    return 'undefined'

df['region'] = df.apply(region, axis=1)
df

CPU times: total: 3.84 s
Wall time: 3.84 s


,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined
...,...,...,...
99995,эльдорадо старый оскол,3705,undefined
99996,frigate для firefox,3630,undefined
99997,укрсиб,3630,undefined
99998,погода в ялте на неделю,3688,undefined


In [290]:
# Проверка
filtered_ = df[df['region'].isin(['Centre','Northwest', 'The Far East'])]
filtered_

,keyword,shows,region
127,авито москва,979292,Centre
370,авито ру санкт петербург,425134,Northwest
564,погода в санкт петербурге,292489,Northwest
849,авито ярославль,209581,Centre
1063,фарпост владивосток,176951,The Far East
...,...,...,...
99590,авито ярославль автомобили с пробегом,3673,Centre
99634,северпост новости мурманской области,3606,Northwest
99808,полармед мурманск запись на прием,3719,Northwest
99890,яндекс метро москва,4548,Centre


In [304]:
# Проверка
df[df['keyword'].str.contains('сахалин', case=False)]['keyword'].unique()

array(['сахалин', 'южно сахалинск',
       'сетевой город образование сахалинская область',
       'сетевой город южно сахалинск', 'погода в южно сахалинске',
       'красная икра сеть магазинов ооо сахалинская рыбная компания'],
      dtype=object)

In [306]:
# Проверка
filtered1 = df[df['region'].isin(['The Far East'])]
filtered1[filtered1['keyword'].str.contains('сахал', case=False)]['keyword'].unique()

array(['сахалин', 'южно сахалинск',
       'сетевой город образование сахалинская область',
       'сетевой город южно сахалинск', 'погода в южно сахалинске',
       'красная икра сеть магазинов ооо сахалинская рыбная компания'],
      dtype=object)